In [1]:
import os
os.environ['https_proxy'] = 'https://172.30.12.56:3128'

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import DataCollector as collector
dc = collector.DataCollector()

dc._init_()
data = dc.loadData()
data = data['CSTARTTIME','CSERVICE','CSENDERENDPOINTID','CSENDERPROTOCOL','CINBOUNDSIZE','year', 'month', 'day', 'hour', 'minute', 'outcome']

In [ ]:
train, test = data.randomSplit([0.85, 0.15], seed=1)

In [ ]:
from pyspark.ml.feature import VectorAssembler

inputColumns = ['CSTARTTIME','CSERVICE','CSENDERENDPOINTID','CSENDERPROTOCOL','CINBOUNDSIZE','year', 'month', 'day', 'hour', 'minute']

feature_cols = data[inputColumns]
featurizer = VectorAssembler(inputCols=feature_cols.columns, outputCol="features")
train_data = featurizer.transform(train).select('features', 'outcome')
test_data = featurizer.transform(test).select('features', 'outcome')

In [ ]:
display(train_data.groupBy("outcome").count().show())

In [ ]:
from synapse.ml.lightgbm import *

model = LightGBMClassifier(
    objective="binary", featuresCol="features", labelCol="outcome", isUnbalance=True
).fit(train_data)

In [ ]:
model = model.fit(train_data)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

feature_importances = model.getFeatureImportances()
fi = pd.Series(feature_importances, index=feature_cols)
fi = fi.sort_values(ascending=True)
f_index = fi.index
f_values = fi.values

# print feature importances
print("f_index:", f_index)
print("f_values:", f_values)

# plot
x_index = list(range(len(fi)))
x_index = [x / len(fi) for x in x_index]
plt.rcParams["figure.figsize"] = (20, 20)
plt.barh(
    x_index, f_values, height=0.028, align="center", color="tan", tick_label=f_index
)
plt.xlabel("importances")
plt.ylabel("features")
plt.show()

In [ ]:
predictions = model.transform(test_data)
predictions.limit(10).toPandas()

In [ ]:
from synapse.ml.train import ComputeModelStatistics

metrics = ComputeModelStatistics(
    evaluationMetric="classification",
    labelCol="outcome",
    scoredLabelsCol="prediction",
).transform(predictions)
display(metrics)